In [6]:
import pymc as pm
import numpy as np
import pandas as pd
import re




In [3]:
df = pd.read_csv('../data/08_combined_modeling_data.csv')
df.head()

C:\Users\ksbuf\AppData\Local\Temp\ipykernel_23168\3218537555.py:1: DtypeWarning: Columns (208) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/08_combined_modeling_data.csv')


,student_number,ac_ind,overall_gpa,days_attended,days_absent,school_membership,percent_days_attended,is_one_percent_y,extended_school_year_y,regular_percent_1.0,...,school_330,school_335,school_406,school_410,school_702,school_703,school_705,school_706,school_710,school_count
0,382716321,0.0,3.908,292,8,300,97.33,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0
1,382716324,1.0,3.606,352,8,360,97.78,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0
2,382716327,1.0,4.000,179,1,180,99.44,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,382716330,1.0,2.901,180,0,180,100.00,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,382716336,1.0,3.994,355,5,360,98.61,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [13]:
#extract all of the columns for the different schools
school_cols = [col for col in df.columns if re.match(r"^school_\d+$", col)]
school_indexes = [df.columns.get_loc(col) for col in school_cols]

In [58]:
# Initialize a dictionary to store the count of students per school
school_advanced_course_counts = {}

# Iterate over the school columns
for col in df.columns:
    if re.match(r"^school_\d+$", col):  # Only check the school columns
        # Count how many students attended the school and took the advanced course
        count = ((df[col] == 1) & (df['ac_ind'] == 1)).sum()
        if count > 0:
            school_advanced_course_counts[col] = count

# Print the schools and how many students have taken the advanced course
for school, count in school_advanced_course_counts.items():
    print(f"School: {school}, Students who took the advanced course: {count}")


School: school_106, Students who took the advanced course: 186
School: school_109, Students who took the advanced course: 207
School: school_111, Students who took the advanced course: 243
School: school_118, Students who took the advanced course: 198
School: school_120, Students who took the advanced course: 231
School: school_124, Students who took the advanced course: 158
School: school_128, Students who took the advanced course: 134
School: school_130, Students who took the advanced course: 159
School: school_132, Students who took the advanced course: 134
School: school_140, Students who took the advanced course: 124
School: school_144, Students who took the advanced course: 290
School: school_152, Students who took the advanced course: 107
School: school_156, Students who took the advanced course: 286
School: school_160, Students who took the advanced course: 206
School: school_164, Students who took the advanced course: 232
School: school_166, Students who took the advanced cour

In [20]:
# Extract key variables
schools = df.iloc[:, school_indexes].values  # Columns indicating school membership
ac = df['ac_ind'].values
gpa = df['overall_gpa'].values
# test_score = df["test_score"].values
# outcome = df["outcome"].values  # Binary outcome (1/0)
num_schools = schools.shape[1]  # Number of school indicators

ac

array([0., 1., 1., ..., 0., 0., 0.])

In [ ]:


with pm.Model() as model:
    # Global intercept (baseline outcome probability)
    alpha = pm.Normal("alpha", mu=0, sigma=10)

    # Hyperpriors for school effects
    school_mu = pm.Normal("school_mu", mu=0, sigma=10)  # Average effect of schools
    school_sigma = pm.HalfNormal("school_sigma", sigma=10)  # Variability across schools

    # School effects (Hierarchical prior)
    school_effects = pm.Normal("school_effects", mu=school_mu, sigma=school_sigma, shape=num_schools)

    # Coefficients for student-level variables
    # beta_gpa = pm.Normal("beta_gpa", mu=3.6, sigma=.3)
    # beta_test = pm.Normal("beta_test", mu=0, sigma=.5)

    # Compute logits using actual data
    logits = (
        alpha +
        (schools @ school_effects)  # School membership effects
        
    ) 

    

    
    # Likelihood (observed outcomes)
    y_obs = pm.Bernoulli("y_obs", logit_p=pm.math.sigmoid(logits), observed=ac)

    # Sample from the posterior
    trace = pm.sample(2000, tune=1000, target_accept=0.9)

    


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, school_mu, school_sigma, school_effects]


c:\Users\ksbuf\anaconda3\envs\pymc_env\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 172 seconds.
There were 1191 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


AttributeError: 'InferenceData' object has no attribute 'point'

In [67]:

# Summarize results
pm.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,-0.378,9.963,-19.121,18.139,0.234,0.166,1819.0,2491.0,1.00
school_mu,8.113,6.200,-1.523,20.315,0.299,0.212,472.0,726.0,1.01
school_effects[0],8.339,12.532,-15.662,33.730,0.300,0.270,1515.0,1983.0,1.01
school_effects[1],8.609,12.081,-13.573,33.491,0.319,0.266,1342.0,1808.0,1.01
school_effects[2],8.445,11.928,-12.826,33.856,0.316,0.261,1247.0,2042.0,1.01
school_effects[3],8.509,12.284,-15.629,31.971,0.297,0.282,1390.0,2008.0,1.00
school_effects[4],8.435,12.204,-12.981,34.730,0.333,0.278,1225.0,1669.0,1.01
school_effects[5],8.445,12.293,-14.002,33.233,0.322,0.271,1215.0,1854.0,1.01
school_effects[6],8.504,12.106,-13.146,34.263,0.308,0.257,1320.0,2167.0,1.00
school_effects[7],8.391,12.143,-15.490,31.728,0.326,0.261,1225.0,1972.0,1.00


In [42]:
model.debug(verbose= True)

point={'alpha': array(0.), 'school_mu': array(0.), 'school_sigma_log__': array(2.30258509), 'school_effects': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}

The variable y_obs has the following parameters:
0: Sigmoid [id A] <Vector(float64, shape=(18474,))>
 └─ Sigmoid [id B] <Vector(float64, shape=(18474,))>
    └─ Add [id C] <Vector(float64, shape=(18474,))>
       ├─ ExpandDims{axis=0} [id D] <Vector(float64, shape=(1,))>
       │  └─ alpha [id E] <Scalar(float64, shape=())>
       └─ CGemv{no_inplace} [id F] <Vector(float64, shape=(18474,))>
          ├─ AllocEmpty{dtype='float64'} [id G] <Vector(float64, shape=(18474,))>
          │  └─ 18474 [id H] <Scalar(int64, shape=())>
          ├─ 1.0 [id I] <Scalar(float64, shape=())>
          ├─ [[0. 0. 0. ... 1. 0. 0.]] [id J] <Matrix(float64, shape=(18474, 27))>
          ├─ school_effects [id K] <Vector(float64, shape=(27,))>
          └─ 0.0 [id L] <Scalar(